# Objective (shortened)

# Libraries

In [1]:
import pandas as pd 
import datetime as dt 
import pandas_datareader.data as web
import numpy as np

# Data Cleaning Function

In [115]:
#function for cleaning data
def data(excel):
    excel['Ticker'] = excel.Ticker.str.split(' ').str[0]
    return excel[['Ticker','Market Cap','P/B']]

#creating the databases for the 4 screener excel sheets
sm_val,lg_val = data(pd.read_excel('factors.xlsx',sheet_name="sm_val_mom")),data(pd.read_excel('factors.xlsx',sheet_name="lg_val_mom"))
sm_gro,lg_gro = data(pd.read_excel('factors.xlsx',sheet_name="sm_growth_mom")),data(pd.read_excel('factors.xlsx',sheet_name="lg_growth_mom"))

# Adj Close Price Query  / Average Return Over Two Years Function

In [107]:
#queries all the returns for the 2 years from 2017 to 2019 based on size, value, and momentum
def total_returns(df):
    failed = [] #failed list
    passed = [] #passed list
    data = pd.DataFrame() #create a dataframe for the loop
    for ticker in df['Ticker'].values: #loop through all the tickers 
        try: #try clause
            data[ticker] = web.DataReader(ticker, "yahoo", '2017-01-01', '2019-01-01')["Adj Close"] #2 year tick price data
            passed.append(ticker) #append passed tickers to the list
        except (IOError, KeyError): #error
            failed.append(ticker) #failed tickers
    return data, (((data.pct_change()*100).sum())/2).mean(),passed,failed #average two yearly return of all found securities in the database

In [96]:
#wound up saving each query to an excel within this directory to save time (each query took a while due to high number of securities)

# Yearly Return of Each Portfolio

In [92]:
def yearly_return(df):
    #average yearly return of each portfolio
    return (((df.pct_change(len(df)-1).iloc[-1]).mean())/2)*100

In [93]:
sm_val = yearly_return(pd.read_excel('small_value_momentum.xlsx').set_index('Date'))
lg_val = yearly_return(pd.read_excel('large_value_momentum.xlsx').set_index('Date'))
sm_gro = yearly_return(pd.read_excel('small_growth_momentum.xlsx').set_index('Date'))
lg_gro = yearly_return(pd.read_excel('large_growth_momentum.xlsx').set_index('Date'))
sp500 = yearly_return(web.DataReader('^GSPC','yahoo','2017-01-01', '2019-01-01')["Adj Close"])

In [106]:
print(f'''Small Cap, Value, Momentum:{sm_val}%
Large, Value, Momentum: {lg_val}%
Small, Growth, Momentum {sm_gro}%
Large, Growth, Momentum: {lg_gro}%
Benchmark: {sp500}%''')

Small Cap, Value, Momentum:21.747149669781376%
Large, Value, Momentum: 2.9896943716334445%
Small, Growth, Momentum 12.585663424911111%
Large, Growth, Momentum: 11.746914336649999%
Benchmark: 5.514587256673609%


# Check Without Momentum

In [116]:
smval = data(pd.read_excel('data_three_factor.xlsx',sheet_name="sm_value"))
lgval = data(pd.read_excel('data_three_factor.xlsx',sheet_name="lg_val"))
lggro = data(pd.read_excel('data_three_factor.xlsx',sheet_name="lg_gro"))
smgro = data(pd.read_excel('data_three_factor.xlsx',sheet_name="sm_gro"))

In [119]:
one = total_returns(smval)
two = total_returns(lgval)
three = total_returns(lggro)
four = total_returns(smgro)

In [122]:
norm_smval = yearly_return(one[0])
norm_lgval = yearly_return(two[0])
norm_lggro = yearly_return(three[0])
norm_smgro = yearly_return(four[0])

In [127]:
print(f'''Small Cap, Value:{norm_smval}%
Large, Value: {norm_lgval}%
Small, Growth: {norm_smgro}%
Large, Growth: {norm_lggro}%
Benchmark: {sp500}%''')

Small Cap, Value:7.711328405806696%
Large, Value: 2.9016630550295206%
Small, Growth: 12.748179184542053%
Large, Growth: 11.958615232932287%
Benchmark: 5.514587256673609%
